In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset.dataset_utils import *
from utils.runctx import RunCtx

In [3]:
run_ctx = RunCtx("<YOUR_TOKEN>")

In [4]:
all_reviews_df = load_reviews(run_ctx.data_dir + "/us_reviews.csv")
# we split review in two parts - before and after acquisition
samples = split_reviews_by_event_date(all_reviews_df)
# we generate small train and validate datasets to be able to do a manual quality evalutaion
train, validate = prepare_train_validate(samples, size=25)

In [5]:
async def async_run_pipeline(model, prompt_fn, dataset, output_name):
    """
    This function runs a specified prompt and model with review_topic_pipeline against the batch of reviews
    :param model: model to use 
    :param prompt_fn: prompt to use
    :param dataset: dataset to run on
    :param output_name: base name for results files
    """
    from pipeline.pipeline import async_review_topic_pipeline

    pipeline_fn = async_review_topic_pipeline(model, prompt_fn)
    results = await run_ctx.run_async(pipeline_fn, dataset)
    run_ctx.state.save(results, output_name + ".json", overwrite=True)
    pd.DataFrame.from_records(sum(results, [])).to_csv("%s/results_%s.csv" % (run_ctx.data_dir, output_name), index=False)
    return results

In [10]:
from prompts.prompts import make_extraction_prompt_v2
results_sample = await async_run_pipeline(run_ctx.model_gpt3_5, make_extraction_prompt_v2, train, "results_sample_gpt_3_5_final")

In [7]:
# here we prepare a dataset containing equals number of reviews made before and after acquisition
all_reviews = prepare_stratified_dataset(samples)

In [9]:
results_all = await async_run_pipeline(run_ctx.model_gpt3_5, make_extraction_prompt_v2, all_reviews[0:300], "results_all_gpt_3_5_final")